In [1]:
import keras
import pandas as pd
from keras import layers
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
import numpy as np

2025-01-05 01:09:40.035278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736039380.045535  294408 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736039380.048472  294408 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 01:09:40.059380: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Dense, InputLayer, Bidirectional,LSTM, Dropout, SimpleRNN, GRU
# from keras import ops
import numpy as np

In [3]:
DC_POWER_INDEX = 0
LEARNING_RATE = 0.001
RANDOM_STATE = 44
START_TIME = '2020-06-07 00:30:00'
END_TIME = '2020-06-07 04:00:00'
NUMBER_OF_FEATURES = 8
BATCH_SIZE = 1024
# Training window
MULTIV_TRAINING_WINDOW = 8

def fix_randomness():
    tf.random.set_seed(RANDOM_STATE)

    from numpy.random import seed
    seed(RANDOM_STATE)
    keras.utils.set_random_seed(RANDOM_STATE)

fix_randomness()

MULTIVARIATE

In [4]:
EPOCHS = 500
MULTIV_TRAINING_WINDOW = 3
lead_time = 40

In [5]:
# Creating the model input for multivariate forecasting
def df_to_model_input2(df_np, col_index, window_size):
    df_np = df_np.to_numpy()
    x = []
    y = []

    for i in range(len(df_np) - window_size - lead_time):
        row = [r for r in df_np[i:i+window_size]]
        x.append(row)

        label = df_np[i+lead_time+window_size][col_index]
        y.append(label)

    return np.array(x), np.array(y)

In [6]:
multiv_teacher_path = 'models/girasol/multiv_teacher_model.keras'
multiv_student_path = 'models/girasol/multiv_student_model.keras'
distilled_student_path = 'models/girasol/distilled_multiv_student_model.keras'

In [7]:
train = pd.read_csv('/.../girasol_met/train.csv')
test = pd.read_csv('/.../girasol_met/val.csv')

In [8]:
NUMBER_OF_FEATURES = len(train.columns)

In [9]:
scaler = MinMaxScaler()
train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)
test = pd.DataFrame(scaler.transform(test), columns=test.columns)

In [10]:
def get_train_test_val(x, y):
    return x[:train_len], y[:train_len], x[train_len:], y[train_len:]

x, y = df_to_model_input2(train, 7, MULTIV_TRAINING_WINDOW)

ds_len = len(y)
train_len = int(0.8*ds_len)

x_train, y_train, x_val, y_val = get_train_test_val(x, y)
x_test, y_test = df_to_model_input2(test, 7, MULTIV_TRAINING_WINDOW)

In [11]:
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def calculate_mape(scaler, y_pred_scaled, target_column_index=-1):

    # Create dummy arrays to match the scaler's expected input shape
    dummy_data_pred = np.zeros((y_pred_scaled.shape[0], NUMBER_OF_FEATURES))
    dummy_data_test = np.zeros((y_test.shape[0], NUMBER_OF_FEATURES))

    # Insert the predicted values and y_test into the correct column
    dummy_data_pred[:, target_column_index] = y_pred_scaled[:, 0]
    dummy_data_test[:, target_column_index] = y_test[:] # it kept that way to be able to modify if needed

    # Perform inverse transform to get the unscaled predictions and y_test
    unscaled_predictions = scaler.inverse_transform(dummy_data_pred)[:, target_column_index]
    y_test_unscaled = scaler.inverse_transform(dummy_data_test)[:, target_column_index]



    mape = mean_absolute_percentage_error(y_test_unscaled, unscaled_predictions)
    return mape

def calculate_metrics(y_true, y_pred):
    """
    Calculate NMSE, NRMSE, and NMAE normalized by the average of true values.
    
    Parameters:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.
        
    Returns:
        dict: A dictionary containing NMSE, NRMSE, and NMAE.
    """
    # Convert inputs to numpy arrays for consistency
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Mean Squared Error (MSE) and Mean Absolute Error (MAE) using sklearn
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    mape = calculate_mape(scaler=scaler, y_pred_scaled=y_pred)*100
    # Variance and mean of true values
    avg_true = np.mean(y_true)
    
    # Calculate metrics
    nrmse = np.sqrt(mse) / avg_true
    nmae = mae / avg_true

    print(f"NRMSE (Normalized Root Mean Squared Error): {nrmse:.4f}")
    print(f"NMAE (Normalized Mean Absolute Error): {nmae:.4f}")
    print(f"MAPE (Mean Absolute Percentage Error): {mape:.4f}") 
    
    # Return metrics as a dictionary
    return nrmse, nmae, mape

In [12]:
columns = ['Hidden Units', 'Dense Units', 'RMSE']
results_df = pd.DataFrame(columns=columns)

In [13]:
from sklearn.preprocessing import MinMaxScaler
import os

# Check if the 'results' directory exists in the current working directory
if not os.path.exists('results'):
    # If the directory does not exist, you can create it
    os.makedirs('results')

script_name = 'results/multiv_girasol_analysis'
def run_student_multiv(hidden_unit, dense_param):

    global results_df  # To modify the global DataFrame

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(LSTM(hidden_unit))
    model.add(Dropout(0.5))
    model.add(Dense(dense_param, 'relu'))
    model.add(Dense(1, 'linear'))

    # Model checkpoint
    model_file = multiv_student_path

    cp = ModelCheckpoint(model_file, save_best_only=True)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    # Compile the model
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=[MeanSquaredError()]
    )
   
    # Train the model
    model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[cp, early_stopping]
    )

    # Make predictions on the scaled test set
    y_pred = model.predict(x_test)

    nrmse, nmae, mape = calculate_metrics(y_test, y_pred)

    # Add a new row to the DataFrame with all metrics
    new_row = pd.DataFrame({
        'Hidden Units': [hidden_unit], 
        'Dense Units': [dense_param], 
        'NRMSE': [nrmse], 
        'NMAE': [nmae], 
        'MAPE': [mape]
    })

    # Concatenate the new row to the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Save the DataFrame to a CSV file after each run
    results_df.to_csv(f'{script_name}.csv', index=False)

    calculate_mape(scaler=scaler, y_pred_scaled=y_pred)

    # Return a summary string with all the metrics
    return f"Hidden Units: {hidden_unit}, Dense Units: {dense_param}, NRMSE: {nrmse:.5f}, NMAE: {nmae:.5f}, MAPE: {mape:.5f}"


hidden_units = [16, 32, 64, 128, 512]
dense_params = [8, 16]

rmse_list = []
for i in hidden_units:
    for j in dense_params:
        RANDOM_STATE = 44
        tf.random.set_seed(RANDOM_STATE)
        from numpy.random import seed
        seed(RANDOM_STATE)
        keras.utils.set_random_seed(RANDOM_STATE)

        s = run_student_multiv(i,j)
        rmse_list.append(s)

rmse_list

I0000 00:00:1736039381.957716  294408 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/500


I0000 00:00:1736039383.466701  294493 cuda_dnn.cc:529] Loaded cuDNN version 90300


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.2290 - mean_squared_error: 0.2289 - val_loss: 0.0280 - val_mean_squared_error: 0.0280
Epoch 2/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0519 - mean_squared_error: 0.0519 - val_loss: 0.0245 - val_mean_squared_error: 0.0257
Epoch 3/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0352 - mean_squared_error: 0.0352 - val_loss: 0.0217 - val_mean_squared_error: 0.0230
Epoch 4/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0180 - val_mean_squared_error: 0.0193
Epoch 5/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0245 - mean_squared_error: 0.0245 - val_loss: 0.0141 - val_mean_squared_error: 0.0154
Epoch 6/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0212 - mean_squared_error: 0.0212 - val_loss: 0.0117 - val_mean_squared_error: 0.0129
Epoch 7/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0190 - mean_squared_error: 0.0190 - val_loss: 0.0110 - val_mean_squa

['Hidden Units: 16, Dense Units: 8, NRMSE: 0.16710, NMAE: 0.10625, MAPE: 15.48296',
 'Hidden Units: 16, Dense Units: 16, NRMSE: 0.33461, NMAE: 0.28833, MAPE: 28.01613',
 'Hidden Units: 32, Dense Units: 8, NRMSE: 0.16632, NMAE: 0.10622, MAPE: 13.69648',
 'Hidden Units: 32, Dense Units: 16, NRMSE: 0.18142, NMAE: 0.13188, MAPE: 15.00753',
 'Hidden Units: 64, Dense Units: 8, NRMSE: 0.16831, NMAE: 0.11097, MAPE: 13.97817',
 'Hidden Units: 64, Dense Units: 16, NRMSE: 0.17482, NMAE: 0.12098, MAPE: 14.24287',
 'Hidden Units: 128, Dense Units: 8, NRMSE: 0.16485, NMAE: 0.10239, MAPE: 13.11608',
 'Hidden Units: 128, Dense Units: 16, NRMSE: 0.16280, NMAE: 0.10004, MAPE: 12.84819',
 'Hidden Units: 512, Dense Units: 8, NRMSE: 0.16471, NMAE: 0.10003, MAPE: 12.88326',
 'Hidden Units: 512, Dense Units: 16, NRMSE: 0.16554, NMAE: 0.10117, MAPE: 13.10639']

In [14]:
rmse_list

['Hidden Units: 16, Dense Units: 8, NRMSE: 0.16710, NMAE: 0.10625, MAPE: 15.48296',
 'Hidden Units: 16, Dense Units: 16, NRMSE: 0.33461, NMAE: 0.28833, MAPE: 28.01613',
 'Hidden Units: 32, Dense Units: 8, NRMSE: 0.16632, NMAE: 0.10622, MAPE: 13.69648',
 'Hidden Units: 32, Dense Units: 16, NRMSE: 0.18142, NMAE: 0.13188, MAPE: 15.00753',
 'Hidden Units: 64, Dense Units: 8, NRMSE: 0.16831, NMAE: 0.11097, MAPE: 13.97817',
 'Hidden Units: 64, Dense Units: 16, NRMSE: 0.17482, NMAE: 0.12098, MAPE: 14.24287',
 'Hidden Units: 128, Dense Units: 8, NRMSE: 0.16485, NMAE: 0.10239, MAPE: 13.11608',
 'Hidden Units: 128, Dense Units: 16, NRMSE: 0.16280, NMAE: 0.10004, MAPE: 12.84819',
 'Hidden Units: 512, Dense Units: 8, NRMSE: 0.16471, NMAE: 0.10003, MAPE: 12.88326',
 'Hidden Units: 512, Dense Units: 16, NRMSE: 0.16554, NMAE: 0.10117, MAPE: 13.10639']